In [27]:
#!/usr/bin/env python3
import re
import pandas as pd
import ast

# Lista (o set) de ingredientes objetivo (coincidencia en minúsculas)
TARGET_INGREDIENTS = {
    "bacon", "garden cress-chamsur ko saag-", "green lentils", "chicken gizzards",
    "red beans", "pumpkin -farsi-", "sajjyun -moringa drumsticks-", "chickpeas",
    "green brinjal", "buff meat", "ham", "butter", "mutton", "papaya", "paneer",
    "broccoli", "mayonnaise", "rahar ko daal", "soyabean -bhatmas-", "kimchi",
    "beaten rice -chiura-", "bethu ko saag", "sugar", "ketchup", "thukpa noodles",
    "cauliflower", "sausage", "cornflakec", "noodle", "chowmein noodles", "salt",
    "chili powder", "palak -indian spinach-", "moringa leaves -sajyun ko munta-",
    "soy sauce", "milk", "green soyabean -hariyo bhatmas-", "tori ko saag",
    "chicken", "beef", "olive oil", "seaweed", "tofu", "black beans", "minced meat",
    "green peas", "crab meat", "strawberry", "ginger", "ice", "water melon",
    "wallnut", "long beans -bodi-", "yellow lentils", "pea", "orange", "fish",
    "apple", "pear", "wheat"
}

In [28]:
def clean_recipe_name(name: str) -> str:
    """
    Limpia el nombre de la receta para que sea más legible.
    - Convierte a minúsculas
    - Elimina contenido entre paréntesis ( ... )
    - Elimina contenido entre corchetes [ ... ] (por si existe)
    - Elimina caracteres no deseados, dejando letras, dígitos, espacios y guiones
    - Elimina espacios repetidos
    """
    name = name.lower()
    # Eliminar contenido entre paréntesis
    name = re.sub(r"\([^)]*\)", "", name)
    # Eliminar contenido entre corchetes
    name = re.sub(r"\[[^]]*\]", "", name)
    # Mantener solo letras, dígitos, espacios y guiones
    name = re.sub(r"[^a-z0-9\s-]", "", name)
    # Remover espacios repetidos
    name = re.sub(r"\s+", " ", name).strip()
    return name

In [29]:

def has_target_ingredient(ing_list):
    """
    Retorna True si la receta contiene al menos uno de los ingredientes objetivo,
    comparando en minúsculas.
    """
    ing_list_lower = [i.strip().lower() for i in ing_list]
    return any(ing in TARGET_INGREDIENTS for ing in ing_list_lower)


In [30]:
def main():
    # 1. Cargar el CSV RAW_recipes.csv
    df = pd.read_csv("foodcom_data/RAW_recipes.csv")
    print("Total de filas en RAW_recipes:", len(df))

    # 2. Eliminar filas sin 'name', 'ingredients' o 'nutrition'
    df = df.dropna(subset=["name", "ingredients", "nutrition"])

    # 3. Convertir la columna 'ingredients' a lista Python
    df["ingredient_list"] = df["ingredients"].apply(ast.literal_eval)

    # 4. Filtrar recetas que contengan al menos uno de los ingredientes objetivo
    df_filtered = df[df["ingredient_list"].apply(has_target_ingredient)]
    print("Recetas tras filtrar por ingredientes objetivo:", len(df_filtered))

    # 5. Seleccionar aleatoriamente 20,000 recetas (o menos si hay menos disponibles)
    n_muestras = min(len(df_filtered), 20000)
    df_sample = df_filtered.sample(n=n_muestras, random_state=42)
    print("Recetas seleccionadas:", len(df_sample))

    # 6. Convertir la columna 'nutrition' a lista de valores
    df_sample["nutrition_list"] = df_sample["nutrition"].apply(ast.literal_eval)

    # 7. Separar la lista nutricional en 7 columnas
    nutri_cols = [
        "calories",
        "total_fat_pdv",
        "sugar_pdv",
        "sodium_pdv",
        "protein_pdv",
        "saturated_fat_pdv",
        "carbs_pdv"
    ]
    nutrition_df = pd.DataFrame(df_sample["nutrition_list"].tolist(), index=df_sample.index)
    nutrition_df.columns = nutri_cols

    # 8. Crear DataFrame final con name + columnas nutricionales
    #    y limpiar el nombre de la receta.
    df_sample["clean_name"] = df_sample["name"].apply(clean_recipe_name)

    # Construir DataFrame final
    df_final = pd.concat([df_sample["clean_name"], nutrition_df], axis=1).reset_index(drop=True)

    # 9. Generar IDs nuevos del 1 al número de recetas
    df_final.index = df_final.index + 1
    df_final.insert(0, "id", df_final.index)

    # 10. Guardar el nuevo CSV
    df_final.to_csv("new_recipes_20000.csv", index=False)
    print("Nuevo CSV guardado: new_recipes_20000.csv")

if __name__ == "__main__":
    main()


Total de filas en RAW_recipes: 231637
Recetas tras filtrar por ingredientes objetivo: 174209
Recetas seleccionadas: 20000
Nuevo CSV guardado: new_recipes_20000.csv


In [31]:
#!/usr/bin/env python3
"""
Script para generar un CSV de tripletas (ternas) a partir de dos fuentes:
1. new_recipes_20000.csv: contiene, para cada receta (con un ID nuevo y un nombre normalizado),
   los valores nutricionales separados en 7 columnas.
2. RAW_recipes.csv: contiene la columna "ingredients" (en forma de cadena de lista).

Para cada receta, el script genera:
  - Una tripleta "has_ingredient" por cada ingrediente presente.
  - Una tripleta para cada campo nutricional, usando las siguientes reglas de clasificación:
  
    - calories:      low_calories (<174), normal_calories (174–520), high_calories (>520)
    - total_fat_pdv: low_fat (<8), normal_fat (8–41), high_fat (>41)
    - sugar_pdv:     low_sugar (<9), normal_sugar (9–68), high_sugar (>68)
    - sodium_pdv:    low_sodium (<5), normal_sodium (5–33), high_sodium (>33)
    - protein_pdv:   low_protein (<7), normal_protein (7–51), high_protein (>51)
    - saturated_fat_pdv: low_saturated_fat (<7), normal_saturated_fat (7–52), high_saturated_fat (>52)
    - carbs_pdv:     low_carbs (<4), normal_carbs (4–16), high_carbs (>16)

El CSV resultante (new_triplets20.csv) tendrá las columnas:
    subject, relation, object

Los "subject" serán el nombre normalizado de la receta (generado a partir de "name").
"""

import pandas as pd
import ast
import re


In [32]:

# --- Función para limpiar/normalizar el nombre de la receta ---
def clean_recipe_name(name: str) -> str:
    # Convertir a minúsculas
    name = name.lower()
    # Eliminar contenido entre paréntesis y corchetes
    name = re.sub(r"\([^)]*\)", "", name)
    name = re.sub(r"\[[^]]*\]", "", name)
    # Mantener solo letras, dígitos, espacios y guiones
    name = re.sub(r"[^a-z0-9\s-]", "", name)
    # Reducir espacios y hacer strip
    name = re.sub(r"\s+", " ", name).strip()
    return name


In [33]:

# --- Función para clasificar valores nutricionales ---
def classify_value(value, thresholds, low_label, normal_label, high_label):
    """
    thresholds: tuple (low_threshold, high_threshold)
      - Si value < low_threshold -> low_label
      - Si low_threshold <= value <= high_threshold -> normal_label
      - Si value > high_threshold -> high_label
    """
    low_th, high_th = thresholds
    if value < low_th:
        return low_label, value
    elif value <= high_th:
        return normal_label, value
    else:
        return high_label, value

# --- Definir reglas nutricionales ---
nutri_rules = {
    'calories': ((174, 520), "low_calories", "normal_calories", "high_calories"),
    'total_fat_pdv': ((8, 41), "low_fat", "normal_fat", "high_fat"),
    'sugar_pdv': ((9, 68), "low_sugar", "normal_sugar", "high_sugar"),
    'sodium_pdv': ((5, 33), "low_sodium", "normal_sodium", "high_sodium"),
    'protein_pdv': ((7, 51), "low_protein", "normal_protein", "high_protein"),
    'saturated_fat_pdv': ((7, 52), "low_saturated_fat", "normal_saturated_fat", "high_saturated_fat"),
    'carbs_pdv': ((4, 16), "low_carbs", "normal_carbs", "high_carbs")
}


In [34]:

def main():
    # --- Parte A: Procesar new_recipes_20000.csv para la información nutricional ---
    # Cargar el CSV que contiene el nombre normalizado y valores nutricionales
    df_new = pd.read_csv("new_recipes_20000.csv")
    # Se asume que este CSV tiene las columnas:
    #   id, clean_name, calories, total_fat_pdv, sugar_pdv, sodium_pdv, protein_pdv, saturated_fat_pdv, carbs_pdv
    # (Si la columna con el nombre normalizado se llama de otra forma, ajústala.)
    
    # --- Parte B: Procesar RAW_recipes.csv para obtener los ingredientes ---
    df_raw = pd.read_csv("foodcom_data/RAW_recipes.csv")
    df_raw = df_raw.dropna(subset=["name", "ingredients"])
    df_raw["ingredient_list"] = df_raw["ingredients"].apply(ast.literal_eval)
    df_raw["clean_name"] = df_raw["name"].apply(clean_recipe_name)
    
    # --- Unir la información: mantener solo recetas de df_new que aparezcan en RAW_recipes ---
    # Se hace un merge por 'clean_name'
    df_merge = pd.merge(df_new, df_raw[["clean_name", "ingredient_list"]], on="clean_name", how="inner")
    # En caso de duplicados, se elimina para tener una única receta por clean_name
    df_merge = df_merge.drop_duplicates(subset=["clean_name"])
    
    print("Recetas en merge (con ingredientes):", len(df_merge))
    
    # --- Generar Tripletas ---
    triples = []

    # 1. Tripletas para "has_ingredient"
    # Por cada receta, cada ingrediente de la lista se genera como una triple
    for _, row in df_merge.iterrows():
        recipe = row["clean_name"]  # Usamos el nombre normalizado como subject
        for ing in row["ingredient_list"]:
            if isinstance(ing, str):
                ingredient = ing.strip().lower()
                triples.append([recipe, "has_ingredient", ingredient])
    
    # 2. Tripletas para las relaciones nutricionales
    # Se generan a partir de las columnas nutricionales del CSV new_recipes_20000.csv (df_new)
    # Nota: Usamos df_new (no el merge) para asegurar que usamos toda la información nutricional
    for _, row in df_new.iterrows():
        recipe = row["clean_name"]
        for col, rule in nutri_rules.items():
            thresholds, low_label, normal_label, high_label = rule
            try:
                value = float(row[col])
            except (ValueError, TypeError):
                continue
            relation_label, val = classify_value(value, thresholds, low_label, normal_label, high_label)
            triples.append([recipe, relation_label, str(val)])
    
    # --- Guardar las tripletas en un nuevo CSV ---
    df_triples = pd.DataFrame(triples, columns=["subject", "relation", "object"])
    df_triples.to_csv("new_triplets20.csv", index=False)
    print("Nuevo CSV de tripletas generado: new_triplets20.csv")

if __name__ == "__main__":
    main()


Recetas en merge (con ingredientes): 19982
Nuevo CSV de tripletas generado: new_triplets20.csv


In [35]:

# --- Definir reglas nutricionales ---
nutri_rules = {
    'calories': ((174, 520), "low_calories", "normal_calories", "high_calories"),
    'total_fat_pdv': ((8, 41), "low_fat", "normal_fat", "high_fat"),
    'sugar_pdv': ((9, 68), "low_sugar", "normal_sugar", "high_sugar"),
    'sodium_pdv': ((5, 33), "low_sodium", "normal_sodium", "high_sodium"),
    'protein_pdv': ((7, 51), "low_protein", "normal_protein", "high_protein"),
    'saturated_fat_pdv': ((7, 52), "low_saturated_fat", "normal_saturated_fat", "high_saturated_fat"),
    'carbs_pdv': ((4, 16), "low_carbs", "normal_carbs", "high_carbs")
}

def main():
    # --- Parte A: Procesar new_recipes_20000.csv para la información nutricional ---
    # Cargar el CSV que contiene el nombre normalizado y valores nutricionales
    df_new = pd.read_csv("new_recipes_20000.csv")
    # Se asume que este CSV tiene las columnas:
    #   id, clean_name, calories, total_fat_pdv, sugar_pdv, sodium_pdv, protein_pdv, saturated_fat_pdv, carbs_pdv
    # (Si la columna con el nombre normalizado se llama de otra forma, ajústala.)
    
    # --- Parte B: Procesar RAW_recipes.csv para obtener los ingredientes ---
    df_raw = pd.read_csv("foodcom_data/RAW_recipes.csv")
    df_raw = df_raw.dropna(subset=["name", "ingredients"])
    df_raw["ingredient_list"] = df_raw["ingredients"].apply(ast.literal_eval)
    df_raw["clean_name"] = df_raw["name"].apply(clean_recipe_name)
    
    # --- Unir la información: mantener solo recetas de df_new que aparezcan en RAW_recipes ---
    # Se hace un merge por 'clean_name'
    df_merge = pd.merge(df_new, df_raw[["clean_name", "ingredient_list"]], on="clean_name", how="inner")
    # En caso de duplicados, se elimina para tener una única receta por clean_name
    df_merge = df_merge.drop_duplicates(subset=["clean_name"])
    
    print("Recetas en merge (con ingredientes):", len(df_merge))
    
    # --- Generar Tripletas ---
    triples = []

    # 1. Tripletas para "has_ingredient"
    # Por cada receta, cada ingrediente de la lista se genera como una triple
    for _, row in df_merge.iterrows():
        recipe = row["clean_name"]  # Usamos el nombre normalizado como subject
        for ing in row["ingredient_list"]:
            if isinstance(ing, str):
                ingredient = ing.strip().lower()
                triples.append([recipe, "has_ingredient", ingredient])
    
    # 2. Tripletas para las relaciones nutricionales
    # Se generan a partir de las columnas nutricionales del CSV new_recipes_20000.csv (df_new)
    # Nota: Usamos df_new (no el merge) para asegurar que usamos toda la información nutricional
    for _, row in df_new.iterrows():
        recipe = row["clean_name"]
        for col, rule in nutri_rules.items():
            thresholds, low_label, normal_label, high_label = rule
            try:
                value = float(row[col])
            except (ValueError, TypeError):
                continue
            relation_label, val = classify_value(value, thresholds, low_label, normal_label, high_label)
            triples.append([recipe, relation_label, str(val)])
    
    # --- Guardar las tripletas en un nuevo CSV ---
    df_triples = pd.DataFrame(triples, columns=["subject", "relation", "object"])
    df_triples.to_csv("new_triplets20.csv", index=False)
    print("Nuevo CSV de tripletas generado: new_triplets20.csv")

if __name__ == "__main__":
    main()


Recetas en merge (con ingredientes): 19982
Nuevo CSV de tripletas generado: new_triplets20.csv
